# Demo script



Import and examine the version of the Azure ML SDK. This demo was created using the 0.1.65 version of the SDK.

In [3]:
import azureml
print("SDK Version: ", azureml.core.__version__)

SDK Version:  0.1.65


## Azure ML Workspaces

A workspace is a ... TODO

This would be a good time to graphically explore the ```TechSummit``` workspace using Visual Studio Code and the Azure ML extension. You can easily see the different kinds of resources that are there. Note that the Visual Studio Code Azure ML extension makes it easy to work with Azure ML workspaces directly from the editor that you're using to create your code. All of the operations that you can do in Visual Studio Code, including examining experiments etc. can be also done _programmatically_ in Azure Notebooks using the AML SDK.

Before we can interact with a workspace, we first need to acquire a reference to it. Create or open the ```TechSummit``` Azure ML workspace. Before you run this, make sure that you **run the Data/Connect to Azure command** from the Jupyter menu above. This will ensure that your Azure login credentials are made available to the code running in this notebook. If you see a device login prompt, this almost always means that you forgot to run the Data/Connect to Azure command.

In [4]:
ws = Workspace.create(name='TechSummit', 
                      subscription_id='15ae9cb6-95c1-483d-a0e3-b1a1a3b06324', 
                      resource_group='juliademo', 
                      location='eastus2',
                      exist_ok=True)
ws.name

'TechSummit'

Before we can submit jobs to the cluster that we saw earlier, we need to retrieve a reference to it. Note that the ```nc6cluster``` has already been created earlier in this demo. The ```ComputeTarget.create``` method can be called to either create or retrieve a reference to an existing cluster; it will not return an error if the cluster already exists.


In [7]:
from azureml.core.compute import BatchAiCompute, ComputeTarget

provisioning_config = BatchAiCompute.provisioning_configuration(vm_size = "STANDARD_NC6",
                                                                autoscale_enabled = True,
                                                                cluster_min_nodes = 10, 
                                                                cluster_max_nodes = 20)

cluster = ComputeTarget.create(ws, 
                               name = "nc6cluster", 
                               provisioning_configuration=provisioning_config)

## Running a simple experiment using our Azure Batch AI cluster

Let's begin by creating a new experiment called ```TechSummitDemo``` in our workspace, and running the ```mnist_with_summaries.py``` file using it. An **Experiment** constructs a reproducible execution environment for running your model training scripts. It does so by creating a **Docker container** for the execution environment for the experiment. The Docker container is saved to the Azure Container Registry in your **Workspace**, ensuring that you will always be able to come back to reproduce the results of this experiment in the future.

Once the container is created, it must be downloaded onto each of the compute nodes on the cluster before the runs can begin. Note that this can take quite some time to complete. But once complete, you now can recreate the environment rapidly by re-running the experiment.

In [10]:
from azureml.core import Experiment
from azureml.train.dnn import TensorFlow

experiment = Experiment(ws, "TechSummitDemo")

estimator = TensorFlow(source_directory='.',
                       compute_target=cluster,
                       entry_script='mnist_with_summaries.py')

run = experiment.submit(estimator)

Once the experiment is _running_, we can use a Jupyter widget to visualize the progress of the run.

In [13]:
from azureml.train.widgets import RunDetails
RunDetails(run).show()

_UserRun()